<a href="https://colab.research.google.com/github/brucker3/CS5560_proejct/blob/main/Topic_modelling_docsummary.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install -U pyLDAvis
!pip install -U gensim

Requirement already up-to-date: pyLDAvis in /usr/local/lib/python3.7/dist-packages (3.3.1)
Requirement already up-to-date: gensim in /usr/local/lib/python3.7/dist-packages (4.0.1)


In [2]:
import re
import string
import nltk
nltk.download('stopwords')
nltk.download('punkt')
from gensim import corpora, models, similarities
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
import pandas as pd
import numpy as np
import time
import gensim
import pyLDAvis
import pyLDAvis.gensim_models as gensimvis
from gensim.models import CoherenceModel

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


/usr/local/lib/python3.7/dist-packages/gensim/similarities/__init__.py:15: UserWarning: The gensim.similarities.levenshtein submodule is disabled, because the optional Levenshtein package <https://pypi.org/project/python-Levenshtein/> is unavailable. Install Levenhstein (e.g. `pip install python-Levenshtein`) to suppress this warning.
  warnings.warn(msg)
/usr/local/lib/python3.7/dist-packages/past/types/oldstr.py:5: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated since Python 3.3,and in 3.9 it will stop working
  from collections import Iterable
/usr/local/lib/python3.7/dist-packages/past/builtins/misc.py:4: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated since Python 3.3,and in 3.9 it will stop working
  from collections import Mapping
/usr/local/lib/python3.7/dist-packages/sklearn/decomposition/_lda.py:29: DeprecationWarning: `np.float` is a deprecate

In [3]:
#read the csv file with amazon reviews
# with open('/content/SpanishArticle.txt', 'r') as f:
#     text = f.readlines()
with open('./docsummary.txt', 'r') as f:
    meta = f.readlines()

text = meta[0].split('.')
reviews_df = pd.DataFrame(text, columns=['text'])
# reviews_df=pd.read_csv('/content/SpanishArticle.txt',error_bad_lines=False)
print(reviews_df.head(6))
print("----------")
print(reviews_df.dtypes)
print("----------")
print(reviews_df.shape)


                                                text
0  Often, 85% amino acid sequence similarity is u...
1   Similarly, the T cell repertoire can be asses...
2   Nevertheless, bulk sequencing of receptors fr...
3   This has been most evident in the clinical di...
4   Furthermore, sequencing can be used to track ...
5   For example, influenza vaccine responses are ...
----------
text    object
dtype: object
----------
(50, 1)


In [4]:
def initial_clean(text):
    """
    Function to clean text-remove punctuations, lowercase text etc.
    """
    text = re.sub("[^a-zA-Z ]", "", text)
    text = text.lower()  # lower case text
    text = nltk.word_tokenize(text)
    return (text)

In [5]:
stop_words = stopwords.words('english')
# adding some more stop words that doesn't convey much meaning in terms of reviews feel free to extend or reduce this list
stop_words.extend(['news', 'say','use', 'not', 'would', 'say', 'could', '_', 'be', 'know', 
                   'good', 'go', 'get', 'do','took','time','year',
                 'done', 'try', 'many', 'some','nice', 'thank', 'think', 'see', 'rather',
                  'easy', 'easily', 'lot', 'lack', 'make', 'want', 'seem', 'run', 'need', 
                   'even', 'right', 'line','even', 'also', 'may', 'take', 'come',
                   'new','said', 'like','people'])

In [6]:
def remove_stop_words(text):
     return [word for word in text if word not in stop_words]

In [7]:
stemmer = PorterStemmer()
def stem_words(text):
    """
    Function to stem words
    """
    try:
        text = [stemmer.stem(word) for word in text]
        text = [word for word in text if len(word) > 1] # no single letter words
    except IndexError:
        pass

    return text

In [8]:
def apply_all(text):
    """
    This function applies all the functions above into one
    """
    return stem_words(remove_stop_words(initial_clean(text)))

print(apply_all)

<function apply_all at 0x7f6e1df213b0>


In [9]:
t1 = time.time()
reviews_df['tokenized_reviews'] = reviews_df['text'].apply(apply_all)
t2 = time.time()
print("Time to clean and tokenize", len(reviews_df), "reviews:", (t2-t1)/60, "min") #Time to clean and tokenize 3209 reviews: 0.21254388093948365 min

print('\n')
print("reviews with their respective tokenize version:" )
print(reviews_df.head(5))

Time to clean and tokenize 50 reviews: 0.001107168197631836 min


reviews with their respective tokenize version:
                                                text                                  tokenized_reviews
0  Often, 85% amino acid sequence similarity is u...  [often, amino, acid, sequenc, similar, use, si...
1   Similarly, the T cell repertoire can be asses...  [similarli, cell, repertoir, assess, sequenc, ...
2   Nevertheless, bulk sequencing of receptors fr...  [nevertheless, bulk, sequenc, receptor, cell, ...
3   This has been most evident in the clinical di...             [evid, clinic, diagnosi, cell, cancer]
4   Furthermore, sequencing can be used to track ...  [furthermor, sequenc, use, track, expans, neoa...


In [10]:
#LDA
#Create a Gensim dictionary from the tokenized data
tokenized = reviews_df['tokenized_reviews']
#Creating term dictionary of corpus, where each unique term is assigned an index.
dictionary = corpora.Dictionary(tokenized)
#Filter terms which occurs in less than 1 review and more than 80% of the reviews.
dictionary.filter_extremes(no_below=1, no_above=0.8)
#convert the dictionary to a bag of words corpus
corpus = [dictionary.doc2bow(tokens) for tokens in tokenized]
print(corpus[:1])

print([[(dictionary[id], freq) for id, freq in cp] for cp in corpus[:1]])

[[(0, 1), (1, 1), (2, 1), (3, 1), (4, 1), (5, 1), (6, 1), (7, 1), (8, 1)]]
[[('acid', 1), ('amino', 1), ('criterion', 1), ('inclus', 1), ('often', 1), ('sequenc', 1), ('similar', 1), ('simpl', 1), ('use', 1)]]


In [11]:
#LDA with num_topics=10, alpha=0.05, eta=0.61, num_words=5 
ldamodel = gensim.models.ldamodel.LdaModel(corpus, num_topics = 5, id2word=dictionary, passes=7, alpha=0.05, eta=0.61)

#saving the model
ldamodel.save('model_combined.gensim')
topics = ldamodel.print_topics(num_words=5)
print('\n')
print("Now printing the topics and their composition")
print("This output shows the Topic-Words matrix for the 10 topics created and the 5 words within each topic")
for topic in topics:
   print(topic)




Now printing the topics and their composition
This output shows the Topic-Words matrix for the 10 topics created and the 5 words within each topic
(0, '0.010*"provid" + 0.010*"level" + 0.010*"converg" + 0.010*"evid" + 0.010*"specif"')
(1, '0.031*"cell" + 0.027*"sequenc" + 0.019*"clone" + 0.015*"tcr" + 0.015*"bcr"')
(2, '0.023*"cell" + 0.023*"sequenc" + 0.018*"repertoir" + 0.018*"immun" + 0.018*"group"')
(3, '0.044*"et" + 0.044*"al" + 0.021*"studi" + 0.014*"vaccin" + 0.014*"converg"')
(4, '0.032*"sequenc" + 0.015*"popul" + 0.015*"immun" + 0.015*"repertoir" + 0.015*"vaccin"')


In [12]:
# Compute Coherence Score for num_topics=10, alpha=0.05, eta=0.61, num_words=5
coherence_model_lda = CoherenceModel(model=ldamodel, texts=tokenized, dictionary=dictionary, coherence='c_v')
coherence_lda = coherence_model_lda.get_coherence()
print('\nCoherence Score: ', coherence_lda)


Coherence Score:  0.41323931943899017


In [13]:
#finding the similarity of the first sentence with topics for num_topics=10, alpha=0.05, eta=0.61, num_words=5
print('\n')
print("first line is:")
print(reviews_df.text[0])
get_document_topics = ldamodel.get_document_topics(corpus[0])
print('\n')
print("The similarity of this line with the topics and respective similarity score are ")
print(get_document_topics)



first line is:
Often, 85% amino acid sequence similarity is used as a simple inclusion criterion


The similarity of this line with the topics and respective similarity score are 
[(4, 0.97837794)]


In [14]:
#visualizing topics for alpha=0.05, eta=0.61
lda_viz = gensim.models.ldamodel.LdaModel.load('/content/model_combined.gensim')
lda_display = pyLDAvis.gensim_models.prepare(lda_viz, corpus, dictionary, sort_topics=True)
pyLDAvis.display(lda_display)
